In [1]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 3.9 MB/s eta 0:00:00


In [2]:
# Import necessary libraries
import optuna
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load the Pima Indian Diabetes dataset from sklearn
# Note: Scikit-learn's built-in 'load_diabetes' is a regression dataset.
# We will load the actual diabetes dataset from an external source
import pandas as pd

# Load the Pima Indian Diabetes dataset (from UCI repository)
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
columns = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI',
           'DiabetesPedigreeFunction', 'Age', 'Outcome']

# Load the dataset
df = pd.read_csv(url, names=columns)

df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
import numpy as np

# Replace zero values with NaN in columns where zero is not a valid value
cols_with_missing_vals = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
df[cols_with_missing_vals] = df[cols_with_missing_vals].replace(0, np.nan)

# Impute the missing values with the mean of the respective column
df.fillna(df.mean(), inplace=True)

# Check if there are any remaining missing values
print(df.isnull().sum())


Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64


In [4]:
# Split into features (X) and target (y)
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# Split data into training and test sets (70% train, 30% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Optional: Scale the data for better model performance
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Check the shape of the data
print(f'Training set shape: {X_train.shape}')
print(f'Test set shape: {X_test.shape}')


Training set shape: (537, 8)
Test set shape: (231, 8)


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [6]:
# Create a study object and optimize the objective function
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters


[I 2024-10-24 09:34:07,208] A new study created in memory with name: no-name-c05ed254-9c78-42d8-974a-15ee503fd544
[I 2024-10-24 09:34:08,389] Trial 0 finished with value: 0.7746741154562383 and parameters: {'n_estimators': 137, 'max_depth': 19}. Best is trial 0 with value: 0.7746741154562383.
[I 2024-10-24 09:34:09,018] Trial 1 finished with value: 0.7783985102420857 and parameters: {'n_estimators': 75, 'max_depth': 20}. Best is trial 1 with value: 0.7783985102420857.
[I 2024-10-24 09:34:11,293] Trial 2 finished with value: 0.7783985102420856 and parameters: {'n_estimators': 196, 'max_depth': 17}. Best is trial 1 with value: 0.7783985102420857.
[I 2024-10-24 09:34:12,957] Trial 3 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 119, 'max_depth': 12}. Best is trial 1 with value: 0.7783985102420857.
[I 2024-10-24 09:34:14,329] Trial 4 finished with value: 0.7765363128491621 and parameters: {'n_estimators': 118, 'max_depth': 18}. Best is trial 1 with value: 0.77839

In [7]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 119, 'max_depth': 20}


In [8]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.75


## Samplers in Optuna

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Define the objective function
def objective(trial):
    # Suggest values for the hyperparameters
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)

    # Create the RandomForestClassifier with suggested hyperparameters
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=42
    )

    # Perform 3-fold cross-validation and calculate accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()

    return score  # Return the accuracy score for Optuna to maximize


In [10]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler())  # We aim to maximize accuracy
study.optimize(objective, n_trials=50)  # Run 50 trials to find the best hyperparameters

[I 2024-10-24 09:35:02,619] A new study created in memory with name: no-name-b598a990-946d-44d6-ba4b-6ce13819df79
[I 2024-10-24 09:35:03,411] Trial 0 finished with value: 0.7635009310986964 and parameters: {'n_estimators': 140, 'max_depth': 10}. Best is trial 0 with value: 0.7635009310986964.
[I 2024-10-24 09:35:03,992] Trial 1 finished with value: 0.7579143389199254 and parameters: {'n_estimators': 103, 'max_depth': 9}. Best is trial 0 with value: 0.7635009310986964.
[I 2024-10-24 09:35:04,379] Trial 2 finished with value: 0.7709497206703911 and parameters: {'n_estimators': 67, 'max_depth': 7}. Best is trial 2 with value: 0.7709497206703911.
[I 2024-10-24 09:35:05,248] Trial 3 finished with value: 0.7597765363128491 and parameters: {'n_estimators': 177, 'max_depth': 4}. Best is trial 2 with value: 0.7709497206703911.
[I 2024-10-24 09:35:05,947] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 141, 'max_depth': 5}. Best is trial 2 with value: 0.770949720

In [11]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7821229050279329
Best hyperparameters: {'n_estimators': 104, 'max_depth': 12}


In [12]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.76


In [13]:
search_space = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [5, 10, 15, 20]
}

In [14]:
# Create a study and optimize it using GridSampler
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.GridSampler(search_space))
study.optimize(objective)

[I 2024-10-24 09:35:38,160] A new study created in memory with name: no-name-6072adae-4a83-48fc-bdb9-b84742705808
[I 2024-10-24 09:35:38,903] Trial 0 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 5}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-10-24 09:35:39,740] Trial 1 finished with value: 0.7672253258845437 and parameters: {'n_estimators': 150, 'max_depth': 10}. Best is trial 0 with value: 0.7690875232774674.
[I 2024-10-24 09:35:40,027] Trial 2 finished with value: 0.7728119180633147 and parameters: {'n_estimators': 50, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-10-24 09:35:40,578] Trial 3 finished with value: 0.7653631284916201 and parameters: {'n_estimators': 100, 'max_depth': 15}. Best is trial 2 with value: 0.7728119180633147.
[I 2024-10-24 09:35:41,166] Trial 4 finished with value: 0.7690875232774674 and parameters: {'n_estimators': 100, 'max_depth': 20}. Best is trial 2 with value: 0.772811

In [15]:

# Print the best result
print(f'Best trial accuracy: {study.best_trial.value}')
print(f'Best hyperparameters: {study.best_trial.params}')

Best trial accuracy: 0.7746741154562384
Best hyperparameters: {'n_estimators': 50, 'max_depth': 5}


In [16]:
from sklearn.metrics import accuracy_score

# Train a RandomForestClassifier using the best hyperparameters from Optuna
best_model = RandomForestClassifier(**study.best_trial.params, random_state=42)

# Fit the model to the training data
best_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = best_model.predict(X_test)

# Calculate the accuracy on the test set
test_accuracy = accuracy_score(y_test, y_pred)

# Print the test accuracy
print(f'Test Accuracy with best hyperparameters: {test_accuracy:.2f}')


Test Accuracy with best hyperparameters: 0.74


## Optuna Visualizations

In [17]:
# For visualizations
from optuna.visualization import plot_optimization_history, plot_parallel_coordinate, plot_slice, plot_contour, plot_param_importances

In [18]:
# 1. Optimization History
plot_optimization_history(study).show()

In [19]:
# 2. Parallel Coordinates Plot
plot_parallel_coordinate(study).show()

In [20]:
# 3. Slice Plot
plot_slice(study).show()

In [21]:
# 4. Contour Plot
plot_contour(study).show()

In [22]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

## Optimizing Multiple ML Models

In [23]:
# Importing the required libraries
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC

In [24]:
# Define the objective function for Optuna
def objective(trial):
    # Choose the algorithm to tune
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'RandomForest', 'GradientBoosting'])

    if classifier_name == 'SVM':
        # SVM hyperparameters
        c = trial.suggest_float('C', 0.1, 100, log=True)
        kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly', 'sigmoid'])
        gamma = trial.suggest_categorical('gamma', ['scale', 'auto'])

        model = SVC(C=c, kernel=kernel, gamma=gamma, random_state=42)

    elif classifier_name == 'RandomForest':
        # Random Forest hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
        bootstrap = trial.suggest_categorical('bootstrap', [True, False])

        model = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            bootstrap=bootstrap,
            random_state=42
        )

    elif classifier_name == 'GradientBoosting':
        # Gradient Boosting hyperparameters
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = GradientBoostingClassifier(
            n_estimators=n_estimators,
            learning_rate=learning_rate,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=42
        )

    # Perform cross-validation and return the mean accuracy
    score = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy').mean()
    return score

In [25]:
# Create a study and optimize it using CmaEsSampler
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

[I 2024-10-24 09:35:52,858] A new study created in memory with name: no-name-ea99ad54-4ce3-4f78-9b6a-5cf202b85f20
[I 2024-10-24 09:35:52,899] Trial 0 finished with value: 0.6983240223463687 and parameters: {'classifier': 'SVM', 'C': 22.963968734967345, 'kernel': 'sigmoid', 'gamma': 'auto'}. Best is trial 0 with value: 0.6983240223463687.
[I 2024-10-24 09:35:56,439] Trial 1 finished with value: 0.7318435754189944 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 221, 'learning_rate': 0.0481031041687366, 'max_depth': 19, 'min_samples_split': 10, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.7318435754189944.
[I 2024-10-24 09:35:59,102] Trial 2 finished with value: 0.7504655493482307 and parameters: {'classifier': 'GradientBoosting', 'n_estimators': 225, 'learning_rate': 0.03294472817981006, 'max_depth': 18, 'min_samples_split': 2, 'min_samples_leaf': 9}. Best is trial 2 with value: 0.7504655493482307.
[I 2024-10-24 09:35:59,487] Trial 3 finished with value: 0.778

In [26]:
# Retrieve the best trial
best_trial = study.best_trial
print("Best trial parameters:", best_trial.params)
print("Best trial accuracy:", best_trial.value)

Best trial parameters: {'classifier': 'SVM', 'C': 0.12798942999380683, 'kernel': 'linear', 'gamma': 'scale'}
Best trial accuracy: 0.7895716945996275


In [27]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_C,params_bootstrap,params_classifier,params_gamma,params_kernel,params_learning_rate,params_max_depth,params_min_samples_leaf,params_min_samples_split,params_n_estimators,state
0,0,0.698324,2024-10-24 09:35:52.861824,2024-10-24 09:35:52.898667,0 days 00:00:00.036843,22.963969,NaN,SVM,auto,sigmoid,NaN,NaN,NaN,NaN,NaN,COMPLETE
1,1,0.731844,2024-10-24 09:35:52.900902,2024-10-24 09:35:56.438988,0 days 00:00:03.538086,NaN,NaN,GradientBoosting,NaN,NaN,0.048103,19.0,7.0,10.0,221.0,COMPLETE
2,2,0.750466,2024-10-24 09:35:56.441070,2024-10-24 09:35:59.102187,0 days 00:00:02.661117,NaN,NaN,GradientBoosting,NaN,NaN,0.032945,18.0,9.0,2.0,225.0,COMPLETE
3,3,0.778399,2024-10-24 09:35:59.103990,2024-10-24 09:35:59.487067,0 days 00:00:00.383077,NaN,False,RandomForest,NaN,NaN,NaN,12.0,8.0,5.0,88.0,COMPLETE
4,4,0.728119,2024-10-24 09:35:59.488904,2024-10-24 09:35:59.518912,0 days 00:00:00.030008,1.218177,NaN,SVM,auto,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,95,0.787709,2024-10-24 09:36:33.870819,2024-10-24 09:36:33.907967,0 days 00:00:00.037148,0.162130,NaN,SVM,scale,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
96,96,0.759777,2024-10-24 09:36:33.909715,2024-10-24 09:36:34.741928,0 days 00:00:00.832213,NaN,True,RandomForest,NaN,NaN,NaN,7.0,4.0,8.0,157.0,COMPLETE
97,97,0.785847,2024-10-24 09:36:34.743754,2024-10-24 09:36:34.781717,0 days 00:00:00.037963,0.212094,NaN,SVM,auto,linear,NaN,NaN,NaN,NaN,NaN,COMPLETE
98,98,0.707635,2024-10-24 09:36:34.783726,2024-10-24 09:36:34.823821,0 days 00:00:00.040095,0.103010,NaN,SVM,scale,poly,NaN,NaN,NaN,NaN,NaN,COMPLETE


In [28]:
study.trials_dataframe()['params_classifier'].value_counts()

,count
params_classifier,
SVM,76
RandomForest,14
GradientBoosting,10


In [29]:
study.trials_dataframe().groupby('params_classifier')['value'].mean()

,value
params_classifier,
GradientBoosting,0.745810
RandomForest,0.770019
SVM,0.775752


In [30]:
# 1. Optimization History
plot_optimization_history(study).show()

In [31]:
# 3. Slice Plot
plot_slice(study).show()

In [32]:
# 5. Hyperparameter Importance
plot_param_importances(study).show()

In [34]:
! pip install optuna-integration[xgboost]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.9/96.9 kB 4.0 MB/s eta 0:00:00


In [35]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score
import numpy as np

# Load the Iris dataset
X, y = load_iris(return_X_y=True)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the objective function for XGBoost
def objective(trial):
    # Hyperparameter search space
    param = {
        'verbosity': 0,
        'objective': 'multi:softprob',
        'num_class': 3,
        'eval_metric': 'mlogloss',  # Ensure that the eval_metric is specified here
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-8, 1.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-8, 1.0, log=True),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'n_estimators': 300,
    }

    # Create DMatrix for XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Define a pruning callback based on evaluation metrics
    pruning_callback = optuna.integration.XGBoostPruningCallback(trial, "eval-mlogloss")  # Match the metric name in the evals list

    # Train the model
    bst = xgb.train(
        param,
        dtrain,
        num_boost_round=300,
        evals=[(dtrain, "train"), (dtest, "eval")],  # Ensure the eval datasets and names are specified
        early_stopping_rounds=30,
        callbacks=[pruning_callback]
    )

    # Predict on the test set
    preds = bst.predict(dtest)
    best_preds = [int(np.argmax(line)) for line in preds]

    # Return accuracy as the objective value
    accuracy = accuracy_score(y_test, best_preds)
    return accuracy

# Create a study with pruning
study = optuna.create_study(direction='maximize', pruner=optuna.pruners.SuccessiveHalvingPruner())
study.optimize(objective, n_trials=50)

# Output the best trial
print(f"Best trial: {study.best_trial.params}")
print(f"Best accuracy: {study.best_value}")


[I 2024-10-24 09:37:49,675] A new study created in memory with name: no-name-ddd32c9a-deae-4561-83b4-a51f7cec7f81


[0]	train-mlogloss:0.95958	eval-mlogloss:0.95751
[1]	train-mlogloss:0.84759	eval-mlogloss:0.84238
[2]	train-mlogloss:0.74711	eval-mlogloss:0.73729
[3]	train-mlogloss:0.67329	eval-mlogloss:0.66326
[4]	train-mlogloss:0.59840	eval-mlogloss:0.58349
[5]	train-mlogloss:0.54435	eval-mlogloss:0.53026
[6]	train-mlogloss:0.48879	eval-mlogloss:0.46989
[7]	train-mlogloss:0.44064	eval-mlogloss:0.41811
[8]	train-mlogloss:0.40444	eval-mlogloss:0.38550
[9]	train-mlogloss:0.37189	eval-mlogloss:0.35398
[10]	train-mlogloss:0.33869	eval-mlogloss:0.31711
[11]	train-mlogloss:0.30890	eval-mlogloss:0.28477
[12]	train-mlogloss:0.28291	eval-mlogloss:0.25613
[13]	train-mlogloss:0.26054	eval-mlogloss:0.23091
[14]	train-mlogloss:0.24057	eval-mlogloss:0.20866
[15]	train-mlogloss:0.22388	eval-mlogloss:0.19261
[16]	train-mlogloss:0.20925	eval-mlogloss:0.17750
[17]	train-mlogloss:0.19379	eval-mlogloss:0.16201
[18]	train-mlogloss:0.17998	eval-mlogloss:0.14679
[19]	train-mlogloss:0.16860	eval-mlogloss:0.13450
[20]	train

[I 2024-10-24 09:38:00,862] Trial 0 finished with value: 1.0 and parameters: {'lambda': 0.03082998019108656, 'alpha': 0.0003014893527204599, 'eta': 0.1137583279687092, 'gamma': 0.0029922016382036656, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.9484268538131451, 'colsample_bytree': 0.6345957944354853}. Best is trial 0 with value: 1.0.


[0]	train-mlogloss:0.91012	eval-mlogloss:0.91505
[1]	train-mlogloss:0.76934	eval-mlogloss:0.77827
[2]	train-mlogloss:0.64378	eval-mlogloss:0.64392
[3]	train-mlogloss:0.56007	eval-mlogloss:0.55306
[4]	train-mlogloss:0.47887	eval-mlogloss:0.46313
[5]	train-mlogloss:0.42537	eval-mlogloss:0.40583
[6]	train-mlogloss:0.37026	eval-mlogloss:0.34559
[7]	train-mlogloss:0.32472	eval-mlogloss:0.29469
[8]	train-mlogloss:0.29418	eval-mlogloss:0.26539
[9]	train-mlogloss:0.26967	eval-mlogloss:0.23822
[10]	train-mlogloss:0.24113	eval-mlogloss:0.20708
[11]	train-mlogloss:0.22008	eval-mlogloss:0.18096
[12]	train-mlogloss:0.20206	eval-mlogloss:0.15820
[13]	train-mlogloss:0.18629	eval-mlogloss:0.14079
[14]	train-mlogloss:0.17550	eval-mlogloss:0.12647
[15]	train-mlogloss:0.16880	eval-mlogloss:0.11757
[16]	train-mlogloss:0.16390	eval-mlogloss:0.11191
[17]	train-mlogloss:0.16027	eval-mlogloss:0.10779
[18]	train-mlogloss:0.15740	eval-mlogloss:0.10293
[19]	train-mlogloss:0.15414	eval-mlogloss:0.09965
[20]	train

[I 2024-10-24 09:38:05,862] Trial 1 finished with value: 1.0 and parameters: {'lambda': 0.007487860714756954, 'alpha': 2.4715049431265673e-06, 'eta': 0.16698817958564227, 'gamma': 3.807600613064139e-08, 'max_depth': 5, 'min_child_weight': 6, 'subsample': 0.9431414444544364, 'colsample_bytree': 0.6816603817604519}. Best is trial 0 with value: 1.0.


...

[0]	train-mlogloss:1.04662	eval-mlogloss:1.04404
[1]	train-mlogloss:0.99953	eval-mlogloss:0.99603


[I 2024-10-24 09:38:48,692] Trial 48 pruned. Trial was pruned at iteration 1.


[0]	train-mlogloss:0.88260	eval-mlogloss:0.87702
[1]	train-mlogloss:0.72987	eval-mlogloss:0.71823


[I 2024-10-24 09:38:48,771] Trial 49 pruned. Trial was pruned at iteration 1.


Best trial: {'lambda': 0.03082998019108656, 'alpha': 0.0003014893527204599, 'eta': 0.1137583279687092, 'gamma': 0.0029922016382036656, 'max_depth': 5, 'min_child_weight': 3, 'subsample': 0.9484268538131451, 'colsample_bytree': 0.6345957944354853}
Best accuracy: 1.0


In [36]:
from optuna.visualization import plot_intermediate_values

# 1. Plot intermediate values during the trials
plot_intermediate_values(study).show()